In [ ]:
# SEP769 - July 27, 2021
# Amir Kamaleddine, Ernani Fantinatti, Mohammed Ibraheem
# Performing feature selection
# https://github.com/efantinatti/MAFAULDA_SEP769


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.models import Model
from sklearn.feature_selection import SelectKBest, chi2
import datetime
import numpy as np

In [ ]:
# For Google Colab only
# Pulling processed data from public folder
# Credit to this stack overflow forum: https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
import gdown
url = 'https://drive.google.com/uc?id=192BrYfD9UXg2sf91t2knFfd4wRdLhqm7' 
output = 'fsDATA.zip'
gdown.download(url, output, quiet=False)

!unzip /content/fsDATA.zip
!rm -rf sample_data


In [ ]:
train_data = np.loadtxt('/content/fsData/train_data.csv', delimiter=",")
test_data = np.loadtxt('/content/fsData/test_data.csv', delimiter=",")
train_label = np.loadtxt('/content/fsData/trian_label.csv', delimiter=",")
test_label = np.loadtxt('/content/fsData/test_label.csv', delimiter=",")

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_data = scaler.fit_transform(train_data)
test_data = scaler.fit_transform(test_data)

In [ ]:
print(train_data.shape,train_label.shape, test_data.shape, test_label.shape)

In [ ]:
selection = SelectKBest(score_func=chi2, k=6)
selection.fit(train_data, train_label)
new_train_data = selection.transform(train_data)
new_test_data = selection.transform(test_data)

In [ ]:
features = ["rotation frequency" , "underhang accelerometer 1", "underhang accelerometer 2", "underhang accelerometer 3","overhang accelerometer 1", "overhang accelerometer 2", "overhang accelerometer 3" ,"microphone"]

In [ ]:
for i in range(0,8):
  print(features[i]+": ", selection.scores_[i])

In [ ]:
new_train_data.shape, new_test_data.shape

In [ ]:
model = Sequential()
model.add(Dense(60, input_dim=8, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_data, train_label, epochs=5, batch_size=50, validation_data=(test_data, test_label), verbose=2)

In [ ]:
test_accuracy = model.evaluate(test_data, test_label)

In [ ]:
model2 = Sequential()
model2.add(Dense(60, input_dim=6, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

In [ ]:
history = model2.fit(new_train_data, train_label, epochs=5, batch_size=50, validation_data=(new_test_data, test_label), verbose=2)

In [ ]:
test_accuracy = model2.evaluate(new_test_data, test_label)